BD SQL & NOSQL Project
======================
### Project based on data from RGPH (recensement général de la population et de l’habitat) in 2014
### DATA SOURCE: [RGPH Website](http://rgphentableaux.hcp.ma)

## Part 03: Migrate From MySQL to MongoDB Database

In [ ]:
# Install dependencies
!pip install pymysql pymongo

In [ ]:
# Import libraries
import pymysql
import pymongo

### Question 01: MongoDB Database Conception
![MongoDB DB Diagram](https://i.imgur.com/jVPgLKR.png)

In [ ]:
#Define Databases Connections

# MySql Database Connection
MYSQL_PARAMS = {
    'host': 'localhost',
    'port': 3306,
    'user': 'root',
    'password': 'ehtp',
    'charset': 'utf8mb4',
    'cursorclass': pymysql.cursors.DictCursor
}
mysql_cnx = pymysql.connect(**MYSQL_PARAMS)
mysql_crs = mysql_cnx.cursor()

# MongoDB Database Connection
MONGODB_PARAMS = {
    'host': 'localhost',
    'port': 27017,
    'username': 'ehtp',
    'password': 'ehtp'
}
mongodb_clt = pymongo.MongoClient(**MONGODB_PARAMS)


In [ ]:
#Creating and Filling Tables
#Fetch tables witch
sql_query = """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'ehtp_rgph'
    AND table_name like 'observation%'
    ORDER BY table_rows
"""
mysql_crs.execute(sql_query)
data_tables = mysql_crs.fetchall()

sql_query = "USE ehtp_rgph;"
mysql_crs.execute(sql_query)


mongodb_clt.drop_database('ehtp_rgph_01')
mongodb_db = mongodb_clt.ehtp_rgph_01


sql_query = """
    SELECT
    pays.CODE code_pays,
    pays.libelle pays,
    regions.CODE code_region,
    regions.libelle region,
    provinces.`code` code_province,
    provinces.libelle province,
    communes.CODE code_commune,
    communes.libelle commune
    FROM communes
    JOIN provinces ON provinces.CODE = communes.province_code
    JOIN regions ON regions.CODE = provinces.region_code
    JOIN pays ON pays.`code` = regions.pays_code
"""
mysql_crs.execute(sql_query)
communes = mysql_crs.fetchall()


for commune in communes:
    code_commune = commune['code_commune']
    commune_to_mongo = {
        'pays': {'code': commune['code_pays'], 'libelle': commune['pays']},
        'region': {'code': commune['code_region'], 'libelle': commune['region']},
        'province': {'code': commune['code_province'], 'libelle': commune['province']},
        'code': code_commune,
        'libelle': commune['commune'],
    }

    for table in data_tables:
        table = table['table_name']
        sql_query = f"""
            SELECT
            indicateur,
            observation
            FROM {table}
            WHERE commune_code = '{code_commune}'
        """
        mysql_crs.execute(sql_query)
        commune_to_mongo[table] = mysql_crs.fetchall()

    mongodb_db.communes.insert_one(commune_to_mongo)